In [132]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split

In [133]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.2.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0 pyspark-shell'
spark = SparkSession.builder.master("local[*]").appName('test').getOrCreate()

In [134]:
from pyspark.ml import PipelineModel
pipeline = PipelineModel.load("pipeline2.model")

In [135]:
df = spark \
    .readStream \
    .format('kafka') \
    .option('kafka.bootstrap.servers', "venomoth.fib.upc.edu:9092") \
    .option('subscribe', 'bdm_p2') \
    .load() \
    .selectExpr("CAST(value AS STRING)")

In [146]:
value_split = df.withColumn('time', split(df['value'], ',').getItem(0)) \
       .withColumn('NeighborhoodId', split(df['value'], ',').getItem(1)) \
       .withColumn('price', split(df['value'], ',').getItem(2)) \
       .select('time', 'NeighborhoodId', 'price') \
       
predict = pipeline.transform(value_split).select("time","NeighborhoodId", "price", "prediction")

predict \
    .writeStream \
    .queryName("predict") \
    .format("memory") \
    .start()

In [154]:
spark.sql("select * from predict").show()

+--------------------+--------------+-------+------------------+
|                time|NeighborhoodId|  price|        prediction|
+--------------------+--------------+-------+------------------+
|2022-05-31 09:49:...|      Q3596096| 249912|258621.65917207283|
|2022-05-31 09:49:...|      Q3753110| 224981| 272000.1218273012|
|2022-05-31 09:50:...|      Q3297896| 899038|1277444.4609025393|
|2022-05-31 09:50:...|      Q2562684| 435100| 2202789.802164891|
|2022-05-31 09:50:...|       Q720994| 498076|  441405.319753192|
|2022-05-31 09:50:...|      Q3310216|3500048| 1284819.069259817|
+--------------------+--------------+-------+------------------+



In [150]:
spark.streams.active[0].stop()